---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
%pwd

'/Users/rob/Google_Drive/Datascience/Coursera/UMich-Intro-Data-Science-in-Python/Week-4-Stats'

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    towns_by_state = {}
    towns = []
    with open('university_towns.txt') as fin:
        for line in fin:
            if '[edit]' in line:
                if len(towns) > 1:
                    towns_by_state[state] = towns
                state = line.split('[')[0].strip()
                towns = []
            elif (line.find(',') != -1) and (line.find('(') != -1) and (line.find(',') < line.find('(')) :
                town = line.split(',')[0].strip()
                towns.append(town)
            elif line.find('(') != -1:
                town = line.split('(')[0].strip()
                towns.append(town)
            elif line.find(',') != -1:
                town = line.split(',')[0].strip()
                towns.append(town)
    state_town_tups = [tuple((state, town)) for state, towns in towns_by_state.items() for town in towns]
    df = pd.DataFrame(data=state_town_tups, columns=['State', 'RegionName'])
    df.State.map(states)
    return df
    

In [4]:
get_list_of_university_towns().head()

,State,RegionName
0,Virginia,Blacksburg
1,Virginia,Bridgewater
2,Virginia,Charlottesville
3,Virginia,Farmville
4,Virginia,Fredericksburg


In [5]:
def gdp_data():
    df = pd.read_excel('gdplev.xls', 
        skiprows=7, 
        usecols=[4, 5, 6], 
        names=['qtr', 'gdp_curr', 'gdp_2009'])
    return df

gdp_data().head()

,qtr,gdp_curr,gdp_2009
0,1947q1,243.1,1934.5
1,1947q2,246.3,1932.3
2,1947q3,250.1,1930.3
3,1947q4,260.3,1960.7
4,1948q1,266.2,1989.5


In [6]:
# better approach
def qtr_to_prd(df):
    df['date'] = pd.to_datetime(df.qtr).dt.to_period('q')
    df['year'] = df['date'].dt.year
    df['qtr'] = df['date'].dt.quarter
    return df

In [7]:
qtr_to_prd(gdp_data()).head()

,qtr,gdp_curr,gdp_2009,date,year
0,1,243.1,1934.5,1947Q1,1947
1,2,246.3,1932.3,1947Q2,1947
2,3,250.1,1930.3,1947Q3,1947
3,4,260.3,1960.7,1947Q4,1947
4,1,266.2,1989.5,1948Q1,1948


In [8]:
# good approach
def qrt_to_dt(df):
    df['date'] = df.qtr.copy()
    df[['year', 'qtr']] = df['qtr'].str.split('q', expand=True)
    df.year = df.year.astype(int)
    return df

In [9]:
def gdp_change(df):
    return df.gdp_2009.diff()

In [10]:
def is_increasing(df):
    return (df['delta'] > 0).astype(float)

In [11]:
df = gdp_data()
df = qtr_to_prd(df).set_index('date')
df['delta'] = gdp_change(df)
df['increasing'] = is_increasing(df)
df.head()

,qtr,gdp_curr,gdp_2009,year,delta,increasing
date,,,,,,
1947Q1,1,243.1,1934.5,1947,NaN,0.0
1947Q2,2,246.3,1932.3,1947,-2.2,0.0
1947Q3,3,250.1,1930.3,1947,-2.0,0.0
1947Q4,4,260.3,1960.7,1947,30.4,1.0
1948Q1,1,266.2,1989.5,1948,28.8,1.0


In [12]:
def recession_prep(df):
    df = qtr_to_prd(df).set_index('date')
    df['delta'] = gdp_change(df)
    df['increasing'] = is_increasing(df)
    df['prv1'] = df['increasing'].shift(1)  # look back one
    df['prv2'] = df['increasing'].shift(2)
    df['curr'] = df['increasing']
    df['nxt1'] = df['increasing'].shift(-1) # look ahead one
    df['nxt2'] = df['increasing'].shift(-2)
    df['start'] = ((df['prv1']==1) & (df['curr']==0) & (df['nxt1']==0)).astype(float)
    df['end']   = ((df['prv2']==0) & (df['prv1']==1) & (df['curr']==1)).astype(float)
    return df

recession_prep(gdp_data()).loc['2007q3':'2010q4']

,qtr,gdp_curr,gdp_2009,year,delta,increasing,prv1,prv2,curr,nxt1,nxt2,start,end
date,,,,,,,,,,,,,
2007Q3,3,14569.7,14938.5,2007,99.8,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
2007Q4,4,14685.3,14991.8,2007,53.3,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
2008Q1,1,14668.4,14889.5,2008,-102.3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2008Q2,2,14813.0,14963.4,2008,73.9,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2008Q3,3,14843.0,14891.6,2008,-71.8,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2008Q4,4,14549.9,14577.0,2008,-314.6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2009Q1,1,14383.9,14375.0,2009,-202.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2009Q2,2,14340.4,14355.6,2009,-19.4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2009Q3,3,14384.1,14402.5,2009,46.9,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0


In [13]:
def get_recessions(df):
    """Return list of tuples containing the starting and ending dates for recessions."""
    
    df = df.loc['2000q1':]
    recessions = []
    is_recession = False
    start, end = None, None
    for index, row in df.iterrows():
        if row['start'] == 1.0:
            start = index
            is_recession = True
        elif is_recession and row['end'] == 1.0:
            end = index
            is_recession = False
            recessions.append((start.strftime('%Yq%q'), end.strftime('%Yq%q')))
    return recessions

get_recessions(recession_prep(gdp_data()))

[('2008q3', '2009q4')]

In [14]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # only one recession during this period, so no need to loop over start, end tuples
    start, end = get_recessions(recession_prep(gdp_data()))[0]
    return start

get_recession_start()

'2008q3'

In [15]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    # only one recession during this period, so no need to loop over start, end tuples
    start, end = get_recessions(recession_prep(gdp_data()))[0]
    return end

get_recession_end()

'2009q4'

In [16]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df = recession_prep(gdp_data())
    start, end = get_recessions(df)[0]
    return df.loc[start:end, 'gdp_2009'].idxmin().strftime('%Yq%q')

get_recession_bottom()

'2009q2'

In [17]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv', 
                 dtype={'RegionID': 'str', 
                        'RegionName': 'str', 
                        'State': 'str',
                        'Metro': 'str',
                        'CountyName': 'str',
                        'SizeRank': 'int'})

    region_id_df = df[['RegionID', 'RegionName', 'State', 'Metro', 'CountyName', 'SizeRank']]
    region_id_df = region_id_df.assign(State=df.State.map(states))
    region_id_df.head()

    df = df.drop(['RegionName', 'State', 'Metro', 'CountyName', 'SizeRank'], axis=1)
    df = pd.melt(df,
                 id_vars=['RegionID'],
                 var_name='Date',
                 value_name='MedianSalesPrice')

    df = df.set_index(pd.to_datetime(df.Date)).sort_index()
    df = df.drop('Date', axis=1)
    df = df.loc['2000-01-01':]

    region_mean_df = df.reset_index().groupby([pd.Grouper(freq='Q', key='Date'), 'RegionID']).mean()

    joined_df = pd.merge(region_id_df, region_mean_df.reset_index(), how='inner', on='RegionID')
    
    # Note: Pivoting on State and RegionName without RegionID reduces the number of rows to 10592.
    #       Only by pivoting on all three and later dropping RegionID can I get the correct number of rows (10730)
    #       Finding this was truely painful.
    pivot_df = joined_df.pivot_table(index=['State', 'RegionID', 'RegionName'], columns='Date', values='MedianSalesPrice')
    pivot_df.columns = [c.lower() for c in pivot_df.columns.to_period('Q').format()]
    pivot_df.index = pivot_df.index.droplevel('RegionID')
    
    return pivot_df

In [18]:
df = convert_housing_data_to_quarters()

In [19]:
df.shape

(10730, 67)

In [20]:
df.head(5)

2000q1        2000q2        2000q3  \
State   RegionName                                                 
Alabama Adamsville      69033.333333  69166.666667  69800.000000   
        Bayou La Batre  44066.666667  44500.000000  44266.666667   
        Chalkville      94100.000000  94433.333333  94433.333333   
        Edgewater                NaN           NaN           NaN   
        Birmingham      54033.333333  54400.000000  54966.666667   

                              2000q4        2001q1        2001q2  \
State   RegionName                                                 
Alabama Adamsville      71966.666667  73466.666667  74000.000000   
        Bayou La Batre  43666.666667  42500.000000  43333.333333   
        Chalkville      94433.333333  96066.666667  97433.333333   
        Edgewater                NaN           NaN           NaN   
        Birmingham      56066.666667  56833.333333  57600.000000   

                              2001q3        2001q4        2002q1  \
State   RegionName                                                 
Alabama Adamsville      73333.333333  73100.000000  73333.333333   
        Bayou La Batre  45433.333333  45400.000000  45400.000000   
        Chalkville      98400.000000  99366.666667  99866.666667   
        Edgewater                NaN           NaN           NaN   
        Birmingham      58433.333333  58700.000000  59500.000000   

                               2002q2    ...            2014q2         2014q3  \
State   RegionName                       ...                                    
Alabama Adamsville       73133.333333    ...      77066.666667   75966.666667   
        Bayou La Batre   45566.666667    ...      49800.000000   51000.000000   
        Chalkville      100733.333333    ...     105233.333333  104333.333333   
        Edgewater                 NaN    ...      46733.333333   44800.000000   
        Birmingham       59866.666667    ...      61733.333333   60900.000000   

                               2014q4         2015q1         2015q2  \
State   RegionName                                                    
Alabama Adamsville       71900.000000   71666.666667   73033.333333   
        Bayou La Batre   51766.666667   50733.333333   50500.000000   
        Chalkville      105866.666667  106966.666667  107666.666667   
        Edgewater        43300.000000   42233.333333   44933.333333   
        Birmingham       59533.333333   59900.000000   61966.666667   

                               2015q3         2015q4         2016q1  \
State   RegionName                                                    
Alabama Adamsville       73933.333333   73866.666667   74166.666667   
        Bayou La Batre   50133.333333   48933.333333   48566.666667   
        Chalkville      109666.666667  111133.333333  111833.333333   
        Edgewater        46033.333333   47500.000000   47933.333333   
        Birmingham       62666.666667   63100.000000   62033.333333   

                               2016q2    2016q3  
State   RegionName                               
Alabama Adamsville       74933.333333   74700.0  
        Bayou La Batre   47833.333333   47400.0  
        Chalkville      114400.000000  115700.0  
        Edgewater        48066.666667   47750.0  
        Birmingham       61633.333333   61250.0  

[5 rows x 67 columns]

In [21]:
# diag 1
# expect: ohio akron  67566.6667 61833.3333
# expect: ohio dayton 57600.0000 50000.0000
convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010q3','2015q2','2016q4']]

2010q3        2015q2  2016q4
State RegionName                                    
Ohio  Akron       67566.666667  61833.333333     NaN
      Dayton      57600.000000  50000.000000     NaN

In [22]:
import re
q5 = convert_housing_data_to_quarters()
pattern = re.compile("\d\d\d\dq[1-4]")
res = 'Columns type test: '
if type(q5.columns) == pd.Index: 
    res += 'Passed' 
    res += '\nColumns names test: '
    if q5.columns.map(pattern.match).astype(bool).all():
        res += 'Passed'
    else:
        res += 'Failed'
        res += "\n Do the column names look like ['2000q1','2000q2'....]"
        res += '\n PS. column names are case sensitive'
else:
    res +='Failed'
    res += "\n Did you remember to change the column names back "
    res += "to strings in the format e.g. Index(['2000q1', '2000q2',...],dtype='object')"
res += "\nState name test: "    
if set(q5.index.get_level_values(0)).issubset(set(states.values())):
    res += 'Passed' 
else:
    res += 'Failed'
    res += "\n Did you map the abbreviated state names to the full names using the precoded states dictionary"
res += "\nShape test: "
if q5.shape == (10730,67):
    res += 'Passed'
else:
    res += 'Failed'

#generate all column names
columns = set(pd.date_range(
    start="2000", end="2017", freq='Q').map(
    lambda x: "{:}q{:}".format(x.year,x.quarter))[:-1])
res += "\nColumn Names test: "
if set(q5.columns)==columns:
    res += 'Passed'
else:
    res += 'Failed'
    res += "\nmissing columns" + str(columns-set(q5.columns))

print(res)

Columns type test: Passed
Columns names test: Passed
State name test: Passed
Shape test: Passed
Column Names test: Passed


In [23]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # group university and non-university towns
    univ_towns = get_list_of_university_towns()
    univ_towns['Location'] = univ_towns[['State', 'RegionName']].apply(tuple, axis=1)

    # get home sales data
    sales_df = convert_housing_data_to_quarters().reset_index()
    sales_df['Location'] = sales_df[['State', 'RegionName']].apply(tuple, axis=1)

    # calculate the ratio of prices from the start to the bottom of the recession
    # smaller values indicates that homes retained their value during the recession
    sales_df['Price_Ratio'] = sales_df.loc[:, get_recession_start()] / sales_df.loc[:, get_recession_bottom()]
    sales_df = sales_df.loc[:, ['Location', 'Price_Ratio']]
    sales_df = sales_df.dropna()

    # home sales data series for university and other towns
    is_univ = sales_df.Location.isin(univ_towns.Location)
    univ_sales_df = sales_df.loc[is_univ]
    other_sales_df = sales_df.loc[~is_univ]

    # compare the means of median home prices
    _, p_value = ttest_ind(other_sales_df.Price_Ratio.values, univ_sales_df.Price_Ratio.values)

    # interpret the results
    if p_value < 0.01:
        # reject null hypothesis --> population means are different
        different = True
        better = 'university town'
    else:
        # accept null hypothesis --> population means are the same
        different = False
        better = 'non-university town'

    # return results
    return different, p_value, better


In [24]:
run_ttest()

(True, 0.0050464756985043489, 'university town')